Deliverable 3: Data Collection

In [1]:
import requests, json
import pandas as pd
api_key = "b298e4516b9540fc8af2b12357116222cac81785"

In [2]:
# Pulling all the ZTCAs in the state of Washington from a list of all ZTCAS per State...
# Source: dataset uploaded by JoeGermuska on Github
url = 'https://gist.githubusercontent.com/JoeGermuska/8845e13fbe420128e50eb74d9f48d68d/raw/6a21a0a416bb8c00a0b9e45b5ef9b6fa22292347/99_state_zcta_xref.csv'
col_names = ["State_code", "State", "ZTCA","num_states"]
df = pd.read_csv(url, index_col=0, names = col_names)

Washington_ZTCAS = []
for index,row in df.iterrows():
  if row["State"] == "WA":
    Washington_ZTCAS.append(row['ZTCA'])

In [3]:
#using US census API to load demographics data
url = "https://api.census.gov/data/2022/acs/acs5/subject?get=NAME,S1901_C01_013E,S1501_C02_009E,S1501_C02_012E,S1501_C02_013E,S1701_C03_001E&for=zip%20code%20tabulation%20area:*&key={0}".format(api_key)
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
response = requests.get(url, headers = headers).json()


#Filter response to only show data for Washington
filtered_response = [row for row in response if row[6] in Washington_ZTCAS]

#convert json response to dataframe
df = pd.DataFrame(filtered_response, columns = ["Name Code","Annual Household Income($)","High school graduate(%)","Population with Bachelor's degree(%)",\
                                                "Population with master's or professional degree(%)","Poverty Level(%)","Zip Code"])
update_df = df.reindex(columns=['Zip Code', 'Annual Household Income($)','High school graduate(%)',"Population with Bachelor's degree(%)",\
                                "Population with master's or professional degree(%)","Poverty Level(%)"])

In [4]:
#using US census API to load more demographics data per zip code
url = "https://api.census.gov/data/2022/acs/acs5?get=NAME,B25077_001E,B01003_001E&for=zip%20code%20tabulation%20area:*&key={0}".format(api_key)
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
response1 = requests.get(url, headers = headers).json()

filtered_response1 = [row for row in response1 if row[3] in Washington_ZTCAS]

#convert json response to dataframe
df1 = pd.DataFrame(filtered_response1, columns = ["Name", "Home Value($)","Total Population","Zip Code"])


In [5]:
# Merging all census data and creating a census dataframe
Census_df = update_df.merge(df1, how='right', on="Zip Code")

#setting "Zip Code" column as integer to facilitate merging of dataframe
Census_df["Zip Code"]= Census_df["Zip Code"].astype('int64')

Census_df.reindex(columns=['Zip Code',"Total Population", "Annual Household Income($)",'High school graduate(%)',"Population with Bachelor's degree(%)",\
                           "Population with master's or professional degree(%)","Poverty Level(%)","Home Value($)"])


,Zip Code,Total Population,Annual Household Income($),High school graduate(%),Population with Bachelor's degree(%),Population with master's or professional degree(%),Poverty Level(%),Home Value($)
0,83824,403,110680,26.7,13.4,21.1,20.8,346400
1,83851,2096,70643,36.7,9.9,3.6,14.3,194100
2,83855,1774,80011,35.7,15.5,10.0,6.3,276400
3,83870,415,76606,19.4,19.8,15.9,12.8,341100
4,97031,19376,117747,20.3,22.5,18.6,5.9,522800
...,...,...,...,...,...,...,...,...
596,99363,96,56952,27.9,13.1,0.0,50.0,227800
597,99371,309,61236,38.0,13.2,5.4,25.9,125000
598,99401,383,76738,22.3,44.7,0.0,0.0,318300
599,99402,1594,124547,25.1,17.7,13.8,6.0,320600


In [6]:
#Loading Dataset from Data.gov that contains all the vehicles registered in the state of Washington
#Source link: https://data.wa.gov/Transportation/Electric-Vehicle-Share-of-New-Registrations/wzin-vviu

file_path = "C:\\Users\\weuri\\Downloads\\ISM 6405\\ProjectDB4\\EV_Data_WA.csv"

# Read the CSV file into a DataFrame
EV_Data= pd.read_csv(file_path)

#Merge Census Dataframe with EV_Data
df = pd.merge(Census_df,EV_Data, how='left', on="Zip Code")
df

,Zip Code,Annual Household Income($),High school graduate(%),Population with Bachelor's degree(%),Population with master's or professional degree(%),Poverty Level(%),Name,Home Value($),Total Population,Transaction Month and Year,Make,Model,Model Year,Vehicle Color,Fuel Type Primary,County,State,2020 GEOID,Transaction Count
0,83824,110680,26.7,13.4,21.1,20.8,ZCTA5 83824,346400,403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,83851,70643,36.7,9.9,3.6,14.3,ZCTA5 83851,194100,2096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,83855,80011,35.7,15.5,10.0,6.3,ZCTA5 83855,276400,1774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,83870,76606,19.4,19.8,15.9,12.8,ZCTA5 83870,341100,415,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,97031,117747,20.3,22.5,18.6,5.9,ZCTA5 97031,522800,19376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134086,99403,82532,31.9,13.7,8.4,17.2,ZCTA5 99403,262400,20425,11/1/2023,CHEVROLET,BOLT EV,2023.0,NaN,Electric,Asotin,WA,5.300396e+10,1.0
134087,99403,82532,31.9,13.7,8.4,17.2,ZCTA5 99403,262400,20425,12/1/2023,CHEVROLET,BOLT EV,2023.0,BLACK,Electric,Asotin,WA,5.300396e+10,1.0
134088,99403,82532,31.9,13.7,8.4,17.2,ZCTA5 99403,262400,20425,12/1/2023,TOYOTA,PRIUS PRIME,2021.0,NaN,Electric,Asotin,WA,5.300396e+10,1.0
134089,99403,82532,31.9,13.7,8.4,17.2,ZCTA5 99403,262400,20425,1/1/2024,CHEVROLET,BOLT EUV,2023.0,BLACK,Electric,Asotin,WA,5.300396e+10,1.0


In [7]:
#Loading Number of EV sales per year per zip code based on EV_Data (used pivot table)
##Source link: https://data.wa.gov/Transportation/Electric-Vehicle-Share-of-New-Registrations/wzin-vviu

file_path = "C:\\Users\\weuri\\Downloads\\ISM 6405\\ProjectDB4\\EV_Sales_Count.csv"
# Read the CSV file into a DataFrame
EV_Sales_count= pd.read_csv(file_path)

#Replacing all empty cell in EV_sales_count by 0
EV_Sales_count.fillna(0, inplace=True)

#Merging EV sales count with main dataframe
#df1 = pd.merge(df,EV_Sales_count,how='left', on="Zip Code")
#df1.fillna(0, inplace=True)
df1 = df.merge(EV_Sales_count, how='right', on="Zip Code")
df1.head()

,Zip Code,Annual Household Income($),High school graduate(%),Population with Bachelor's degree(%),Population with master's or professional degree(%),Poverty Level(%),Name,Home Value($),Total Population,Transaction Month and Year,...,County,State,2020 GEOID,Transaction Count,# Sales 2020,# Sales 2021,# Sales 2022,# Sales 2023,# Sales 2024,Grand Total
0,98001,127394,25.6,20.6,8.1,8.3,ZCTA5 98001,483900,35842,1/1/2020,...,King,WA,5.303303e+10,1.0,48.0,117.0,142.0,248.0,36.0,591
1,98001,127394,25.6,20.6,8.1,8.3,ZCTA5 98001,483900,35842,1/1/2020,...,King,WA,5.303303e+10,1.0,48.0,117.0,142.0,248.0,36.0,591
2,98001,127394,25.6,20.6,8.1,8.3,ZCTA5 98001,483900,35842,1/1/2020,...,King,WA,5.303303e+10,1.0,48.0,117.0,142.0,248.0,36.0,591
3,98001,127394,25.6,20.6,8.1,8.3,ZCTA5 98001,483900,35842,1/1/2020,...,King,WA,5.303303e+10,2.0,48.0,117.0,142.0,248.0,36.0,591
4,98001,127394,25.6,20.6,8.1,8.3,ZCTA5 98001,483900,35842,1/1/2020,...,King,WA,5.303303e+10,1.0,48.0,117.0,142.0,248.0,36.0,591


In [8]:
#Loading Number of EV stations per year per zip code based on dataset from US Department of Energy Website (used pivot table)
#source link: https://afdc.energy.gov/fuels/electricity_locations.html#/find/nearest?fuel=ELEC
file_path = "C:\\Users\\weuri\\Downloads\\ISM 6405\\ProjectDB4\\EV_Stations_Count.csv"

# Read the CSV file into a DataFrame
EV_Stations_Count = pd.read_csv(file_path)
EV_Stations_Count.head()

#Merging EV stations count with main dataframe
#df2 = df1.merge(EV_Stations_Count, how='right', on="Zip Code")
df2 = pd.merge(df1,EV_Stations_Count, how='left', on="Zip Code")
df2['Number of EV Stations'].fillna(0, inplace=True)
df2.head()

,Zip Code,Annual Household Income($),High school graduate(%),Population with Bachelor's degree(%),Population with master's or professional degree(%),Poverty Level(%),Name,Home Value($),Total Population,Transaction Month and Year,...,State,2020 GEOID,Transaction Count,# Sales 2020,# Sales 2021,# Sales 2022,# Sales 2023,# Sales 2024,Grand Total,Number of EV Stations
0,98001,127394,25.6,20.6,8.1,8.3,ZCTA5 98001,483900,35842,1/1/2020,...,WA,5.303303e+10,1.0,48.0,117.0,142.0,248.0,36.0,591,6.0
1,98001,127394,25.6,20.6,8.1,8.3,ZCTA5 98001,483900,35842,1/1/2020,...,WA,5.303303e+10,1.0,48.0,117.0,142.0,248.0,36.0,591,6.0
2,98001,127394,25.6,20.6,8.1,8.3,ZCTA5 98001,483900,35842,1/1/2020,...,WA,5.303303e+10,1.0,48.0,117.0,142.0,248.0,36.0,591,6.0
3,98001,127394,25.6,20.6,8.1,8.3,ZCTA5 98001,483900,35842,1/1/2020,...,WA,5.303303e+10,2.0,48.0,117.0,142.0,248.0,36.0,591,6.0
4,98001,127394,25.6,20.6,8.1,8.3,ZCTA5 98001,483900,35842,1/1/2020,...,WA,5.303303e+10,1.0,48.0,117.0,142.0,248.0,36.0,591,6.0


In [9]:
#Loading voting affiliations data from the State of Washington
file_path = "C:\\Users\\weuri\\Downloads\\ISM 6405\\ProjectDB4\\Election_Results.csv"
# Read the CSV file into a DataFrame
Election_data= pd.read_csv(file_path)

#merge Election data with main dataframe
Final_Project = df2.merge(Election_data, how='right', on="County")

In [10]:
#Deleting unnecessary columns
columns_to_drop = ['Name', 'Vehicle Color','2020 GEOID','Transaction Count']  # List of column names to drop

Final_Project.drop(columns=columns_to_drop, inplace=True)

# Delete rows where 'Zip Code' column is empty
Final_Project.dropna(subset=['Zip Code'], inplace=True)

In [11]:
# Calculate the number of unique zip codes
unique_zip_codes = df2['Zip Code'].nunique()

# Print the number of unique zip codes
print("Number of unique zip codes:", unique_zip_codes)


Number of unique zip codes: 540


In [15]:
Final_Project
#file_path = "C:\\Users\\weuri\\Downloads\\ISM 6405\\ProjectDB4\\Deliverable4Data.csv"
#Final_Project.to_csv(file_path, index=False)

,Zip Code,Annual Household Income($),High school graduate(%),Population with Bachelor's degree(%),Population with master's or professional degree(%),Poverty Level(%),Home Value($),Total Population,Transaction Month and Year,Make,...,Number of EV Stations,Dem %,GRN %,LIB %,PSL %,REP %,SWP %,WRI %,Winner,Total Votes Cast
0,99169,78985,25.4,17.7,7.9,7.1,167500,2494,5/1/2020,CHEVROLET,...,3.0,0.309451,0.003071,0.013988,0.001706,0.666496,0.000853,0.004435,REP,"5,862"
1,99169,78985,25.4,17.7,7.9,7.1,167500,2494,2/1/2021,CHEVROLET,...,3.0,0.309451,0.003071,0.013988,0.001706,0.666496,0.000853,0.004435,REP,"5,862"
2,99169,78985,25.4,17.7,7.9,7.1,167500,2494,6/1/2021,CHEVROLET,...,3.0,0.309451,0.003071,0.013988,0.001706,0.666496,0.000853,0.004435,REP,"5,862"
3,99169,78985,25.4,17.7,7.9,7.1,167500,2494,6/1/2021,TESLA,...,3.0,0.309451,0.003071,0.013988,0.001706,0.666496,0.000853,0.004435,REP,"5,862"
4,99169,78985,25.4,17.7,7.9,7.1,167500,2494,12/1/2021,FORD,...,3.0,0.309451,0.003071,0.013988,0.001706,0.666496,0.000853,0.004435,REP,"5,862"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133214,98953,96901,24.3,14.0,8.5,11.3,349400,7043,12/1/2023,TESLA,...,2.0,0.448879,0.003337,0.014814,0.001601,0.525558,0.000821,0.004990,REP,"96,193"
133215,98953,96901,24.3,14.0,8.5,11.3,349400,7043,12/1/2023,TESLA,...,2.0,0.448879,0.003337,0.014814,0.001601,0.525558,0.000821,0.004990,REP,"96,193"
133216,98953,96901,24.3,14.0,8.5,11.3,349400,7043,1/1/2024,TESLA,...,2.0,0.448879,0.003337,0.014814,0.001601,0.525558,0.000821,0.004990,REP,"96,193"
133217,98953,96901,24.3,14.0,8.5,11.3,349400,7043,1/1/2024,KIA,...,2.0,0.448879,0.003337,0.014814,0.001601,0.525558,0.000821,0.004990,REP,"96,193"
